# Домашнє завдання 5

In [7]:
import numpy as np
import pandas as pd
import os
import shutil
import zipfile

## Отримуємо датасет

копіюємо архів `homework.zip` в Colab

In [12]:
filename = 'homework.zip'

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('')
print(f'zipfile {filename} extracted')

df_list = []

for dir_name in os.listdir('data'):
    if not os.path.isdir(f'data/{dir_name}'):
        continue
    else:
        print(f'Working with dir: {dir_name}')
        for file_name in os.listdir(f'data/{dir_name}'):
            if file_name.endswith('.csv'):
                df_csv = pd.read_csv(f'data/{dir_name}/{file_name}')
                df_csv['action'] = dir_name
                df_list.append(df_csv)
        shutil.rmtree(f'data/{dir_name}')

os.rmdir('data')
df = pd.concat(df_list, ignore_index=True)
df.to_csv('df.csv', index=False)
print('df created and saved to df.csv')
df

zipfile homework.zip extracted
Working with dir: walking
Working with dir: running
Working with dir: idle
Working with dir: stairs
df created


,accelerometer_X,accelerometer_Y,accelerometer_Z,action
0,-0.943315,-14.379576,-3.960009,walking
1,-3.404555,7.900866,11.219229,walking
2,-3.423708,-13.048399,-3.275268,walking
3,-15.519216,-15.423448,-15.006856,walking
4,-2.470816,-7.235277,1.991976,walking
...,...,...,...,...
193855,1.460463,-11.267113,-1.340753,stairs
193856,-3.835511,-12.253524,8.130709,stairs
193857,1.776498,-7.091625,0.636858,stairs
193858,14.164097,-11.554418,5.999870,stairs
